# Módulo: Normalización de Textos
Este notebook contiene las funciones para limpiar datos de texto (quitar tildes, ajustar espacios y mayúsculas) y para aplicar reglas de normalización por departamento según cada archivo.

In [4]:
import pandas as pd

In [5]:
def limpiar_texto_columna(serie):
    """
    Limpia una serie de texto (columna):
    - Quita espacios a los extremos
    - Quita tildes de las vocales
    - Pone la primera letra en mayúscula y el resto en minúscula (Formato Título)
    """
    # 1. Minúsculas y limpieza de espacios al inicio/fin
    s = serie.astype(str).str.lower().str.strip()
    
    # 2. Quitar tildes
    reemplazos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u'}
    for acento, sin_acento in reemplazos.items():
        s = s.str.replace(acento, sin_acento)
        
    # 3. Formato Título y arreglar preposiciones
    s = s.str.title()
    s = s.str.replace(' Del ', ' del ').str.replace(' De ', ' de ')
    
    return s

def normalizar_departamentos(df, tipo_archivo):
    """
    Crea la columna 'departamento_normalizado' con condiciones específicas
    dependiendo del archivo (Producción, Precios, Recaudo).
    """
    df_out = df.copy()
    
    # Identificar la columna que corresponde a departamento
    col_depto = [c for c in df_out.columns if 'departamento' in c.lower()]
    if not col_depto:
        return df_out # Si no hay columna, se retorna el dataframe igual
        
    nombre_col_depto = col_depto[0]
    
    # Limpieza a la columna Original
    df_out[nombre_col_depto] = limpiar_texto_columna(df_out[nombre_col_depto])
    
    # Columna nueva donde se aplican las reglas especiales
    df_out['departamento_normalizado'] = df_out[nombre_col_depto].copy()
    
    # --- APLICAR REGLAS GENERALES ---
    # 1. "que si dice valle, se refiere a valle del Cauca" 
    df_out.loc[df_out['departamento_normalizado'] == 'Valle', 'departamento_normalizado'] = 'Valle del Cauca'
    
    # 2. Agrupamos cualquier Santander (Norte de Santander, Santanderes) en "Santander"
    df_out.loc[df_out['departamento_normalizado'].str.contains('Santander', na=False), 'departamento_normalizado'] = 'Santander'
    
    # --- REGLAS ESPECÍFICAS POR ARCHIVO ---
    if tipo_archivo == 'recaudo':
        # 3. "en recaudo... donde decia la inicial de Bogota... poner Cundinamarca"
        # Convertimos todo lo que empiece con Bogota en Cundinamarca
        df_out.loc[df_out['departamento_normalizado'].str.startswith('Bogota', na=False), 'departamento_normalizado'] = 'Cundinamarca'
        
    return df_out

In [6]:
import pandas_gbq
from google.oauth2 import service_account

def cargar_a_bigquery(dfs_procesados, dataset_id="dataset_Fedepapa"):
    """
    Sube un diccionario de dataframes a Google BigQuery.
    """
    # 1. Archivo de credenciales (que ya está ignorado en Git)
    ruta_llave = "llave_google.json"  
    credenciales = service_account.Credentials.from_service_account_file(ruta_llave)

    # 2. El ID  proyecto
    project_id = "silken-champion-476819-c6"
    
    # 3. Iterar sobre los dataframes y mandarlos uno a uno
    for nombre_tabla, df_final in dfs_procesados.items():
        # Aquí define el destino:
        destino_tabla = f"{dataset_id}.{nombre_tabla.lower()}"
        
        print(f"Subiendo {nombre_tabla.upper()} a BigQuery en {destino_tabla}...")
        
        # Sube la tabla con if_exists='replace' (borra la anterior y la actualiza)
        pandas_gbq.to_gbq(
            dataframe=df_final,
            destination_table=destino_tabla,
            project_id=project_id,
            credentials=credenciales,
            if_exists='replace' # Puedes cambiar a 'append' si sólo quieres añadir
        )
        
        print(f"✅ ¡{nombre_tabla.upper()} guardado con éxito en la nube!")

